In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from dataAnalysis import *
from tools import (
    SimulationConfig, 
    DataLoader,
    GPUMonteCarloSimulator, 
    PINNTrainer, 
    ResultAnalyzer, 
    Visualizer,
    setup_deepxde_backend
)

# 设置DeepXDE后端
setup_deepxde_backend()

print("所有库导入完成，DeepXDE后端设置为PyTorch")

In [ ]:
# 创建模拟配置
config = SimulationConfig(
    space_dims=[20.0, 10.0, 10.0],
    grid_shape=[100, 100, 100],
    pinn_grid_shape=[100, 100, 100],
    source_energy_MeV=30.0,
    n_particles=1000000,  # 可以调整为更小值如100000进行快速测试
    rho_air_kg_m3=1.205,
    mass_energy_abs_coeff=0.001901,
    energy_cutoff_MeV=0.01
)

print(f"配置完成:")
print(f"  空间维度: {config.SPACE_DIMS_np} m")
print(f"  网格形状: {config.GRID_SHAPE_np}")
print(f"  粒子数: {config.N_PARTICLES}")
print(f"  源能量: {config.SOURCE_ENERGY_MeV} MeV")
print(f"  世界边界: {config.WORLD_MIN_np} 到 {config.WORLD_MAX_np}")

In [ ]:
# ===== 数据源选择 =====
USE_EXTERNAL_DATA = True  # 设置为True使用外部字典数据，False使用MC模拟数据

if USE_EXTERNAL_DATA:
    print("=" * 60)
    print("使用外部字典格式数据")
    print("=" * 60)
    
    # TODO: 在这里导入您的外部数据
    # 示例1: 从变量导入
    # external_data_dict = your_external_data_dict
    
    # 示例2: 从文件导入
    # import pickle
    # with open('your_data.pkl', 'rb') as f:
    #     external_data_dict = pickle.load(f)
    
    # 示例3: 创建模拟数据（用于测试）
    data = get_data("DATA.xlsx")

    # 分析data数据的统计信息
    all_targets = [data[i][j][k] for i in range(len(data.keys())) for j in range(len(data[0][1])) for k in range(len(data[0][0]))]
    variance = np.var(all_targets)
    minValue = np.min([i for i in all_targets if i!=0])
    maxValue = np.max(all_targets)

    print("data方差为:",variance,"\ndata最大值为:",maxValue,"\ndata最小值为:",minValue)
    
    print(f"外部数据包含 {len(data)} 个z层")
    
    # 使用字典加载接口
    dose_data = DataLoader.load_dose_from_dict(
        data_dict=data,
        space_dims=config.SPACE_DIMS_np
    )
    
    print("外部数据加载完成:")
    
    # ===== 找到剂量率最大值点作为源位置 =====
    dose_grid = dose_data['dose_grid']
    world_min = dose_data['world_min']
    voxel_size = dose_data['voxel_size']
    
    # 找到最大剂量值的网格索引
    max_dose_flat_idx = np.argmax(dose_grid)
    max_dose_grid_idx = np.unravel_index(max_dose_flat_idx, dose_grid.shape)
    
    # 转换为世界坐标（体素中心）
    mc_source_pos = world_min + np.array(max_dose_grid_idx) * voxel_size + voxel_size / 2.0
    
    print(f"找到最大剂量点:")
    print(f"  网格索引: {max_dose_grid_idx}")
    print(f"  剂量值: {dose_grid[max_dose_grid_idx]:.4e} Gy")
    print(f"  世界坐标: ({mc_source_pos[0]:.3f}, {mc_source_pos[1]:.3f}, {mc_source_pos[2]:.3f}) m")
    print(f"  设为 mc_source_pos")
    
else:
    # 运行GPU蒙特卡洛模拟
    mc_source_pos = np.array([0.0, 0.0, 0.0])  # 源位置

    print("开始GPU蒙特卡洛模拟...")
    simulator = GPUMonteCarloSimulator(config)
    dose_grid_mc_raw = simulator.simulate(mc_source_pos)

    print(f"\n蒙特卡洛模拟完成:")
    print(f"  最大剂量: {np.max(dose_grid_mc_raw):.4e} Gy")
    print(f"  非零剂量点数: {np.sum(dose_grid_mc_raw > 1e-30)}")
    print(f"  数据形状: {dose_grid_mc_raw.shape}")
    
    print("=" * 60)
    print("使用MC模拟数据")
    print("=" * 60)
    
    # 使用MC模拟结果
    dose_data = DataLoader.load_dose_from_numpy(
        dose_array=dose_grid_mc_raw,
        space_dims=config.SPACE_DIMS_np
    )
    
    print("MC数据加载完成:")

print("数据预处理完成:")
print(f"  剂量网格形状: {dose_data['grid_shape']}")
print(f"  空间维度: {dose_data['space_dims']} m")
print(f"  体素尺寸: {dose_data['voxel_size']} m")
print(f"  世界边界: {dose_data['world_min']} 到 {dose_data['world_max']}")

# 无源用positive_only，有源用positive_weighted
# 采样训练数据
sampled_points_xyz, sampled_doses_values, sampled_log_doses_values = DataLoader.sample_training_points(
    dose_data, 
    num_samples=300, 
    sampling_strategy='positive_only'
)

print(f"\n训练数据采样完成:")
print(f"  训练点数: {len(sampled_points_xyz)}")
print(f"  剂量值范围: {np.min(sampled_doses_values):.4e} - {np.max(sampled_doses_values):.4e} Gy")
print(f"  log剂量值范围: {np.min(sampled_log_doses_values):.2f} - {np.max(sampled_log_doses_values):.2f}")

In [ ]:
# 物理参数
physical_params = {
    'rho_material': config.RHO_AIR_kg_m3,
    'mass_energy_abs_coeff': config.MASS_ENERGY_ABS_COEFF_m2_kg
}

# 创建预测网格
pred_x_coords = dose_data['world_min'][0] + (np.arange(config.PINN_GRID_SHAPE_np[0]) + 0.5) * (dose_data['space_dims'][0] / config.PINN_GRID_SHAPE_np[0])
pred_y_coords = dose_data['world_min'][1] + (np.arange(config.PINN_GRID_SHAPE_np[1]) + 0.5) * (dose_data['space_dims'][1] / config.PINN_GRID_SHAPE_np[1])
pred_z_coords = dose_data['world_min'][2] + (np.arange(config.PINN_GRID_SHAPE_np[2]) + 0.5) * (dose_data['space_dims'][2] / config.PINN_GRID_SHAPE_np[2])

XX, YY, ZZ = np.meshgrid(pred_x_coords, pred_y_coords, pred_z_coords, indexing='ij')
prediction_points = np.vstack((XX.ravel(), YY.ravel(), ZZ.ravel())).T


print(f"预测网格创建完成: {prediction_points.shape[0]} 个预测点")

pinn_grid_coords = (pred_x_coords, pred_y_coords, pred_z_coords)

In [ ]:
# 创建PINN训练器（无源PDE）
print("="*60)
print("开始无源PDE PINN训练")
print("="*60)

trainer_nosource = PINNTrainer(physical_params=physical_params)

# 创建PINN模型（无源）
model_nosource = trainer_nosource.create_pinn_model(
    dose_data=dose_data,
    sampled_points_xyz=sampled_points_xyz, 
    sampled_log_doses_values=sampled_log_doses_values, 
    include_source=False,
    network_config={'layers': [3] + [32] * 4 + [1], 'activation': 'tanh'}
)

print("无源PINN模型创建完成")

In [ ]:
# 训练无源PINN模型
trainer_nosource.train(epochs=10000, use_lbfgs=True, loss_weights=[1, 100])

# 获取学习到的参数
learned_k_nosource = trainer_nosource.get_learned_parameters()
print(f"\n无源PINN学习到的参数:")
print(f"  k值: {learned_k_nosource[0]:.6f} (1/m)")
print(f"  理论k值: {config.MASS_ENERGY_ABS_COEFF_m2_kg * config.RHO_AIR_kg_m3:.6f} (1/m)")
print(f"  相对误差: {abs(learned_k_nosource[0] - config.MASS_ENERGY_ABS_COEFF_m2_kg * config.RHO_AIR_kg_m3) / (config.MASS_ENERGY_ABS_COEFF_m2_kg * config.RHO_AIR_kg_m3) * 100:.2f}%")

In [ ]:


# 进行预测
dose_pinn_flat_nosource = trainer_nosource.predict(prediction_points)
dose_grid_pinn_nosource = dose_pinn_flat_nosource.reshape(config.PINN_GRID_SHAPE_np)

print(f"无源PINN预测完成:")
print(f"  预测最大剂量: {np.max(dose_grid_pinn_nosource):.4e} Gy")
print(f"  预测网格形状: {dose_grid_pinn_nosource.shape}")

In [ ]:
# 评估无源PINN结果


evaluation_results_nosource = ResultAnalyzer.evaluate_predictions(
    dose_pinn_grid=dose_grid_pinn_nosource,
    dose_mc_data=dose_data,
    pinn_grid_coords=pinn_grid_coords,
    sampled_points_xyz=sampled_points_xyz,
    sampled_doses_values=sampled_doses_values
)

print(f"\n无源PINN评估结果:")
print(f"  全局平均相对误差: {evaluation_results_nosource['mean_relative_error']:.4%}")
print(f"  全局平均绝对误差: {evaluation_results_nosource['mae']:.4e}")
print(f"  全局均方根误差: {evaluation_results_nosource['rmse']:.4e}")
if 'training_mre' in evaluation_results_nosource:
    print(f"  训练点平均相对误差: {evaluation_results_nosource['training_mre']:.4%}")

In [ ]:
# 可视化无源PINN结果
source_positions = {'mc_source': mc_source_pos}

Visualizer.plot_comparison(
    dose_mc_data=dose_data,
    dose_pinn_grid=dose_grid_pinn_nosource,
    pinn_grid_coords=pinn_grid_coords,
    evaluation_results=evaluation_results_nosource,
    source_positions=source_positions,
    learned_params=learned_k_nosource,
    num_particles=config.N_PARTICLES,
    num_training_points=len(sampled_points_xyz)
)

# 3D可视化
Visualizer.plot_3d_comparison(
    dose_mc_data=dose_data,
    dose_pinn_grid=dose_grid_pinn_nosource,
    source_positions=source_positions,
    learned_params=learned_k_nosource
)

print("无源PDE PINN实验完成！")

In [ ]:
# 创建源参数化PINN
print("\n" + "="*60)
print("开始源参数化 PINN训练")
print("="*60)

trainer_source = PINNTrainer(physical_params=physical_params)

# 创建PINN模型（含源参数化）- 使用改进的初始化
model_source = trainer_source.create_pinn_model(
    dose_data=dose_data,
    sampled_points_xyz=sampled_points_xyz, 
    sampled_log_doses_values=sampled_log_doses_values, 
    include_source=True,
    network_config={'layers': [3] + [64] * 6 + [1], 'activation': 'tanh'},
    source_init_method='weighted_centroid',  # 新增：使用加权质心方法
    prior_knowledge=None  # 可选：如果有先验知识可以传入
)

print("源参数化PINN模型创建完成")

# 保存初始参数用于验证（新增）
initial_params = {
    'k_pinn': trainer_source._get_scalar_value(trainer_source.k_pinn),
    'source_params': {name: trainer_source._get_scalar_value(var) 
                      for name, var in trainer_source.source_params.items()}
}


In [ ]:
# 训练源参数化PINN - 现在会显示参数更新过程
trainer_source.train(
    epochs=10000, 
    use_lbfgs=True, 
    loss_weights=[1, 1000],
    display_every=500  # 每500步显示一次参数更新
)

# 验证参数是否正确更新（如果之前保存了initial_params）
if 'initial_params' in locals():
    trainer_source.validate_parameter_updates(initial_params)

# 获取学习到的参数
learned_k_source, learned_source_params = trainer_source.get_learned_parameters()

print(f"\n源参数化PINN学习到的参数:")
print(f"  k值: {learned_k_source:.6f} (1/m)")
print(f"  理论k值: {config.MASS_ENERGY_ABS_COEFF_m2_kg * config.RHO_AIR_kg_m3:.6f} (1/m)")

print(f"\n学习到的源参数:")
print(f"  源位置: ({learned_source_params['xs']:.3f}, {learned_source_params['ys']:.3f}, {learned_source_params['zs']:.3f}) m")
print(f"  真实源位置: ({mc_source_pos[0]:.3f}, {mc_source_pos[1]:.3f}, {mc_source_pos[2]:.3f}) m")
print(f"  位置误差: {np.linalg.norm([learned_source_params['xs'], learned_source_params['ys'], learned_source_params['zs']] - mc_source_pos):.3f} m")
print(f"  源强度 As: {learned_source_params['As']:.2e}")
print(f"  源尺寸 sigma_s: {learned_source_params['sigma_s']:.3f} m")


# 源参数化PINN预测
dose_pinn_flat_source = trainer_source.predict(prediction_points)
dose_grid_pinn_source = dose_pinn_flat_source.reshape(config.PINN_GRID_SHAPE_np)

print(f"\n源参数化PINN预测完成:")
print(f"  预测最大剂量: {np.max(dose_grid_pinn_source):.4e} Gy")

In [ ]:
# 评估源参数化PINN结果
evaluation_results_source = ResultAnalyzer.evaluate_predictions(
    dose_pinn_grid=dose_grid_pinn_source,
    dose_mc_data=dose_data,
    pinn_grid_coords=pinn_grid_coords,
    sampled_points_xyz=sampled_points_xyz,
    sampled_doses_values=sampled_doses_values
)

print(f"\n源参数化PINN评估结果:")
print(f"  全局平均相对误差: {evaluation_results_source['mean_relative_error']:.4%}")
print(f"  全局平均绝对误差: {evaluation_results_source['mae']:.4e}")
print(f"  全局均方根误差: {evaluation_results_source['rmse']:.4e}")
if 'training_mre' in evaluation_results_source:
    print(f"  训练点平均相对误差: {evaluation_results_source['training_mre']:.4%}")

# 可视化源参数化PINN结果
learned_source_pos = np.array([learned_source_params['xs'], learned_source_params['ys'], learned_source_params['zs']])
source_positions_with_learned = {
    'mc_source': mc_source_pos,
    'learned_source': learned_source_pos
}

Visualizer.plot_comparison(
    dose_mc_data=dose_data,
    dose_pinn_grid=dose_grid_pinn_source,
    pinn_grid_coords=pinn_grid_coords,
    evaluation_results=evaluation_results_source,
    source_positions=source_positions_with_learned,
    learned_params=(learned_k_source, learned_source_params),
    num_particles=config.N_PARTICLES,
    num_training_points=len(sampled_points_xyz)
)

# 3D可视化
Visualizer.plot_3d_comparison(
    dose_mc_data=dose_data,
    dose_pinn_grid=dose_grid_pinn_source,
    source_positions=source_positions_with_learned,
    learned_params=(learned_k_source, learned_source_params)
)

print("源参数化PINN实验完成！")

In [ ]:
# # 演示如何使用外部MC数据
# print("\n" + "="*60)
# print("外部数据使用示例")
# print("="*60)

# # 模拟从外部MC软件加载数据的情况
# print("模拟从外部MC软件加载数据...")

# # 示例：从numpy文件加载
# # dose_data_external = DataLoader.load_dose_from_file(
# #     filepath='external_mc_dose.npy',
# #     space_dims=[20.0, 10.0, 10.0],
# #     file_format='npy',
# #     dose_units='cGy'  # 假设外部数据是cGy单位
# # )

# # 示例：直接从数组加载
# external_dose_array = np.random.exponential(scale=1e-6, size=(50, 50, 50))  # 模拟外部数据
# external_dose_array[25, 25, 25] = 1e-3  # 添加一个高剂量点

# dose_data_external = DataLoader.load_dose_from_array(
#     external_dose_array,
#     space_dims=[10.0, 10.0, 10.0],
#     dose_units='mGy'  # 假设外部数据是mGy单位
# )

# print(f"外部数据加载完成:")
# print(f"  数据形状: {dose_data_external['grid_shape']}")
# print(f"  最大剂量: {np.max(dose_data_external['dose_grid']):.4e} Gy")
# print(f"  单位已转换为: {dose_data_external['dose_units']}")

# # 使用不同的采样策略
# sampled_points_external, sampled_doses_external, sampled_log_doses_external = DataLoader.sample_training_points(
#     dose_data_external, 
#     num_samples=100,
#     sampling_strategy='high_dose'  # 采样高剂量区域
# )

# print(f"外部数据采样完成: {len(sampled_points_external)} 个训练点")
# print("外部数据可以用同样的方式进行PINN训练！")

In [ ]:
# 实验结果总结
print("\n" + "="*80)
print("实验结果总结")
print("="*80)

print(f"\n1. 无源PDE PINN:")
print(f"   - 平均相对误差: {evaluation_results_nosource['mean_relative_error']:.4%}")
print(f"   - 均方根误差: {evaluation_results_nosource['rmse']:.4e}")
print(f"   - 学习到的k值: {learned_k_nosource[0]:.6f} (1/m)")
print(f"   - k值相对误差: {abs(learned_k_nosource[0] - config.MASS_ENERGY_ABS_COEFF_m2_kg * config.RHO_AIR_kg_m3) / (config.MASS_ENERGY_ABS_COEFF_m2_kg * config.RHO_AIR_kg_m3) * 100:.2f}%")

print(f"\n2. 源参数化PINN:")
print(f"   - 平均相对误差: {evaluation_results_source['mean_relative_error']:.4%}")
print(f"   - 均方根误差: {evaluation_results_source['rmse']:.4e}")
print(f"   - 学习到的k值: {learned_k_source:.6f} (1/m)")
print(f"   - k值相对误差: {abs(learned_k_source - config.MASS_ENERGY_ABS_COEFF_m2_kg * config.RHO_AIR_kg_m3) / (config.MASS_ENERGY_ABS_COEFF_m2_kg * config.RHO_AIR_kg_m3) * 100:.2f}%")
print(f"   - 源位置误差: {np.linalg.norm(learned_source_pos - mc_source_pos):.3f} m")

print(f"\n3. 性能对比:")
if evaluation_results_source['mean_relative_error'] < evaluation_results_nosource['mean_relative_error']:
    print("   - 源参数化PINN具有更低的预测误差")
else:
    print("   - 无源PDE PINN具有更低的预测误差")

print(f"\n4. 数据统计:")
print(f"   - 训练点数: {len(sampled_points_xyz)}")
print(f"   - MC粒子数: {config.N_PARTICLES}")
print(f"   - 网格分辨率: {config.GRID_SHAPE_np}")
print(f"   - 空间范围: {config.SPACE_DIMS_np} m")

print(f"\n所有实验完成！新的工具库已成功实现原有功能并提供了更好的灵活性。")